# Laboratorio: Métodos de búsqueda

# Mariano Villanueva Aguilar 14/11/2024

En las clases anteriores creaste códigos para realizar búsquedas aleatorias (Simulated Annealing) y búsquedas dirigidas (Optimización Bayesiana). Estos métodos de búsqueda se utilizan para facilitar el proceso de optimización de funciones objetivos compleja y costosas de computar.

En este laboratorio usaremos el dataset de los diferentes tipos de iris, y sus longitudes y anchos de pétalos y sépalos. Utilizaremos un RandomForest para crear un modelo de clasificación y el métrico F1 para decidir cuál es el mejor modelo de acuerdo a lo que tenemos disponible.

1. Carga el dataset de Iris

In [48]:
from sklearn import datasets
X, y = datasets.load_iris(return_X_y=True)

2. Importa el archivo `Bosque.py`.

Este archivo contiene la función `RegresionBosque`, que recibe:
- X: las características independientes
- y: la variable de respuesta
- árboles: cantidad total de árboles
- profundidad de bosque: niveles de profundidad del bosque

Su salida es:
- modelo: El objeto con el modelo ajustado
- f1: El métrico que califica qué tan bueno es el modelo que se ajustó.


In [51]:
import Bosque
modelo, f1 = Bosque.RegresionBosque(X, y, 10, 3)
f1

0.9444444444444444

### Actividad 1:

Inicializa un espacio con 5 muestras en nuestro dominio de variables independientes:
- árboles: números enteros entre 5 y 50.
- profundidad: números enteros entre 2 y 10

Utiliza optimización Bayesiana para encontrar la combinación de árboles y profundidad que **maximice** el métrico F1.

In [122]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
np.random.seed(10)
ARB = np.random.randint(5,50,5)
PROF= np.random.randint(2,10,5)
ARB,PROF

(array([14, 41, 20,  5, 33]), array([3, 7, 2, 7, 3]))

In [179]:
f1_lista=[]
for i in range (len(PROF)):
    modelo, f1 = Bosque.RegresionBosque(X, y, ARB[i], PROF[i])
    f1_lista.append(f1)
f1_lista


[0.9444444444444444,
 0.9555555555555556,
 0.9333333333333333,
 0.9555555555555556,
 0.9444444444444444]

In [137]:
arb_vec = ARB.reshape([-1, 1])
prof_vec = PROF.reshape([-1, 1])
X_hiperparam = np.hstack((arb_vec, prof_vec))
X_hiperparam

array([[14,  3],
       [41,  7],
       [20,  2],
       [ 5,  7],
       [33,  3]])

In [139]:
kernel=1.0*RBF(length_scale=1)
gp= GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10).fit(X_hiperparam,f1_lista)

In [163]:
x1=np.linspace(0,50,100).reshape([-1,1])
x2=np.linspace(2,10,100).reshape([-1,1])
Matrizmatricial=np.hstack((x1, x2))
y_pred,y_std=gp.predict(Matrizmatricial,return_std=True)
y_pred_high=y_pred+1.96*y_std
i_next=np.argmax(y_pred_high)
new_X=np.vstack((X_hiperparam,X_hiperparam[i_next]))
new_X

array([[14,  3],
       [41,  7],
       [20,  2],
       [ 5,  7],
       [33,  3],
       [14,  3]])

In [167]:
X_hiperparam[i_next]

array([14,  3])

In [181]:
Modelo,f1=Bosque.RegresionBosque(X, y, 14, 3)


In [182]:
f1_lista.append(f1)
f1_lista

[0.9444444444444444,
 0.9555555555555556,
 0.9333333333333333,
 0.9555555555555556,
 0.9444444444444444,
 0.9555555555555556]

### Actividad 2:

Inicializa 2 vectores con posibles valores para las variables independientes:
- árboles: números enteros entre 5 y 50
- profundidad: números enteros entre 2 y 10

Utiliza el algoritmo de Simulated Annealing que siga el siguiente orden:
- Elige un punto de partida para las variables.
- Selecciona al azar una de las dos para modificarlas.
- Selecciona un elemento al azar de la lista que contiene los posibles valores de esa variable.
- Sigue el algoritmo ($p$ y $q$) para decidir si usas esa combinación nueva o si mantienes la anterior.